In [1]:
# ! pip install transformers
# ! pip install streamlit
# ! pip install torch
# ! npm install localtunel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56

In [1]:
%%writefile MLM.py
import streamlit as st
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch


# Define a function to predict masked words
def predict_masked_words(sentence):
    # Tokenize the input sentence
    tokenized_input = tokenizer.encode_plus(sentence, return_tensors="pt", add_special_tokens=True)

    # Get the position of the masked token
    masked_index = torch.where(tokenized_input["input_ids"] == tokenizer.mask_token_id)[1]

    # Predict the masked token
    with torch.no_grad():
        outputs = model(**tokenized_input)

    # Get the logits for the masked token
    predictions = outputs.logits[0, masked_index, :]

    # Get the top predictions
    top_indices = torch.topk(predictions, 5, dim=1).indices[0].tolist()

    # Convert token IDs to actual words
    predicted_tokens = [tokenizer.decode([index]).strip() for index in top_indices]

    return predicted_tokens

# Load pre-trained BERT model and tokenizer

bert_base_uncased = "bert-base-uncased"
bert_large_uncased = "bert-large-uncased"
bert_base_multilingual_uncased = "bert-base-multilingual-uncased"
albert_large = "albert-large-v2"


model_name = st.sidebar.radio(
            "Select the Model 👉",
            key="visibility",
            options=[bert_base_uncased, bert_large_uncased, bert_base_multilingual_uncased, albert_large],)

if model_name == albert_large:
  tokenizer = AlbertTokenizer.from_pretrained(model_name)
  model = AlbertForMaskedLM.from_pretrained(model_name)
else:
  tokenizer = BertTokenizer.from_pretrained(model_name)
  model = BertForMaskedLM.from_pretrained(model_name)

# Streamlit app
st.title("BERT Masked Word Predictor")

st.write("Enter a sentence with a masked word, denoted by [MASK], and See BERT's top 5 words predictions.")

# Text input for the sentence
input_sentence = st.text_input("Input Sentence", "I want to go [MASK].")

if st.button("Predict"):
    if "[MASK]" not in input_sentence:
        st.error("The input sentence must contain the [MASK] token.")
    else:
        # Predict masked words
        predicted_words = predict_masked_words(input_sentence)
        st.write("Predicted words:")
        for i, word in enumerate(predicted_words):
            st.write(f"{i+1}. {word}")


Writing MLM.py


In [11]:
!streamlit run /content/MLM.py   & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.105.9.122



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.9.122:8501

npx: installed 22 in 7.374s
your url is: https://wet-trees-count.loca.lt
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-un